In [1]:
!pip install tsfresh

In [2]:
!pip install numba

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tsfresh
from tqdm import tqdm
import seaborn as sns

In [4]:
def read_signals(filename):
    samples_count = 5000

    c = ['name', 'x', 'y']
    for i in range(0, samples_count):
        c.append(f'v{i}')
    c = c + ['cluster', 'p0', 'p1', 'p2', 'p3']
    types = {col_name: int for col_name in ['name', 'cluster', 'p0', 'p1', 'p2', 'p3']}
    types.update({col_name: np.float32 for col_name in ['x', 'y', *[f'v{i}' for i in range(0, samples_count)]]})
    
    df = pd.read_csv(filename, names=c, dtype=types)
    df = df.set_index('name', drop=True)

    return df


def write_signals(df, filename):
    df.to_csv(filename, header=False)

In [5]:
url='https://drive.google.com/file/d/15kAQDCxV8YzdvIp_6K1_eKbMPtI9z70H/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
#data = pd.read_csv(url)
data = read_signals(url)

In [6]:
import os
os.cpu_count()

2

In [ ]:
singal_columns = [f'v{j}' for j in range(0, 5000)]
dataframes = []
train_df = data[data.cluster != -1]
for i in tqdm(train_df.index.values):
    ar = np.array([[i] * 5000, range(5000), data.loc[i][singal_columns].values]).T
    ts_df = pd.DataFrame(ar, columns=['name', 'time', 'value'])
    dataframes.append(ts_df)
ts_df = pd.concat(dataframes)
good_extract_df = tsfresh.extract_features(ts_df, column_id='name', column_sort='time')

Feature Extraction: 100%|██████████| 109/109 [42:30<00:00, 23.40s/it]


In [8]:
good_extract_df

NameError: ignored

In [ ]:
extract_df = pd.concat([extract_df, null_extract_df])
extract_df.tail()

In [ ]:
data = data.sort_index()
extract_df['x'] = data['x'] / np.linalg.norm(data['x'])
extract_df['y'] = data['y'] / np.linalg.norm(data['y'])
extract_df.tail()

In [ ]:
X = extract_df.fillna(-1).values
# Y = train_df.sort_index().cluster.values

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
params = { 
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
clf = RandomForestClassifier(random_state=42)
clf_random = GridSearchCV(clf, params, cv=cv, verbose=2, scoring='accuracy')

In [ ]:
clf_random.fit(X, Y)

In [ ]:
clf_random.best_params_, clf_random.best_score_

In [ ]:
best_rf = clf_random.best_estimator_

In [ ]:
best_rf.predict(X)

In [ ]:
best_rf.predict(X) == Y

In [ ]:
extract_df['cluster'] = best_rf.predict(X)

In [ ]:
extract_df.to_csv('tsfresh.csv', index=False)

In [ ]:
url='https://drive.google.com/file/d/1MAUfLpiA2jbd2hkFAgXUvYheLg6zOlPh/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
#data = pd.read_csv(url)
extract_df = pd.read_csv(url)
extract_df.head()

In [ ]:
data.shape

In [ ]:
extract_df.shape

In [ ]:
extract_df.tail()

In [ ]:
extract_df.fillna(-1, inplace=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
cv = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)
params = { 
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
clf = RandomForestClassifier(random_state=42)
# cbcf = cb.CatBoostClassifier(iterations=10, learning_rate=0.01, random_seed=42)
clf_random = GridSearchCV(clf, params, cv=cv, verbose=2, scoring='accuracy')

In [ ]:
clf_random.fit(extract_df.drop(columns=['cluster', 'x', 'y']), extract_df['cluster'])

In [ ]:
clf_random.best_params_, clf_random.best_score_

In [ ]:
best_rf = clf_random.best_estimator_

In [ ]:
best_rf.predict(X) == Y

In [ ]:
data['x'] = data['x'] / np.linalg.norm(data['x'])
data['y'] = data['y'] / np.linalg.norm(data['y'])
data.cluster = best_rf.predict(data.drop(columns=['cluster', 'p0', 'p1', 'p2', 'p3', 'x', 'y']).values)
fig, ax = plt.subplots(figsize=(20, 10))
plt.scatter(data.x, data.y, c=data.cluster, cmap='Dark2')
plt.show()